In [13]:
%pip install evaluate dataset transformers datasets accelerate spacy kagglehub

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
import numpy as np
import pandas as pd
import os
os.environ['WANDB_DISABLED'] = 'true'

import spacy
import string
!python -m spacy download en_core_web_lg
import en_core_web_lg
from scipy.sparse import hstack
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

     ---------------------------------------- 0.0/400.7 MB ? eta -:--:--
     ---------------------------------------- 0.0/400.7 MB ? eta -:--:--
     ---------------------------------------- 0.5/400.7 MB 3.3 MB/s eta 0:02:03
     --------------------------------------- 3.4/400.7 MB 10.2 MB/s eta 0:00:39
     - ------------------------------------ 11.5/400.7 MB 21.6 MB/s eta 0:00:19
     - ------------------------------------ 18.6/400.7 MB 25.7 MB/s eta 0:00:15
     -- ----------------------------------- 27.5/400.7 MB 28.8 MB/s eta 0:00:13
     --- ---------------------------------- 34.9/400.7 MB 30.0 MB/s eta 0:00:13
     ---- --------------------------------- 42.2/400.7 MB 31.2 MB/s eta 0:00:12
     ---- --------------------------------- 50.9/400.7 MB 32.1 MB/s eta 0:00:11
     ----- -------------------------------- 55.6/400.7 MB 31.0 MB/s eta 0:00:12
     ----- -------------------------------- 58.2/400.7 MB 31.2 MB/s eta 0:00:11
     ----- -------------------------------- 58.5/400.7


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("anzerone/clickbait-titles-ru")

print("Path to dataset files:", path)

100%|██████████| 113k/113k [00:00<00:00, 521kB/s]

Extracting files...
Path to dataset files: C:\Users\slesa\.cache\kagglehub\datasets\anzerone\clickbait-titles-ru\versions\1


In [16]:
df = pd.read_csv('C:/Users/slesa/.cache/kagglehub/datasets/anzerone/clickbait-titles-ru/versions/1/titles_data.csv', delimiter = ';')
df.columns = ['text', 'label']
df

,text,label
0,Родственник раскрыл настоящую фамилию Пугачёво...,1
1,Предсказания Матроны Московской на 2024-й год:...,1
2,"Пророчество схимонахини Нины об антихристе, ми...",1
3,«Думал об этом»: что Путин сказал о своем прее...,1
4,Путин поручил уведомить россиян об изменениях ...,1
...,...,...
3193,Путин поручил передать Республике Крым все акц...,0
3194,ЕК изучит просьбу Венгрии по нарушению Болгари...,0
3195,"Глава ""Россетей"" доложил Путину о достижении ц...",0
3196,"Платформа ""Мой экспорт"" научит устанавливать д...",0


In [17]:
nlp = spacy.load("en_core_web_lg")
stop_words = nlp.Defaults.stop_words
punctuations = string.punctuation

In [18]:
def spacy_tokenizer(text):
    doc = nlp(text)
    mytokens = [ word.lemma_.lower().strip() for word in doc ]
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]
    text = " ".join(mytokens)
    return text

In [19]:
df['text'] = df['text'].apply(spacy_tokenizer)
df

,text,label
0,родственник раскрыл настоящую фамилию пугачёво...,1
1,предсказания матроны московской на 2024 й год ...,1
2,пророчество схимонахини нины об антихристе мир...,1
3,« думал об этом » что путин сказал о своем пре...,1
4,путин поручил уведомить россиян об изменениях ...,1
...,...,...
3193,путин поручил передать республике крым все акц...,0
3194,ек изучит просьбу венгрии по нарушению болгари...,0
3195,глава россетей доложил путину о достижении цел...,0
3196,платформа мой экспорт научит устанавливать дел...,0


In [20]:
from datasets import Dataset
dataset = Dataset.from_pandas(df)
dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 3198
})

In [21]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")

In [22]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [23]:
tokenized_df = dataset.map(preprocess_function, batched=True)

Map: 100%|██████████| 3198/3198 [00:00<00:00, 39647.27 examples/s]


In [24]:
import evaluate

accuracy = evaluate.load("accuracy")
precision = evaluate.load("precision")
metric_f1 = evaluate.load("f1")

In [25]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    d = {
        **accuracy.compute(predictions=predictions, references=labels),
        **precision.compute(predictions=predictions, references=labels),
        **metric_f1.compute(predictions=predictions, references=labels)
    }
    return d

In [26]:
id2label = {0: "не кликбейт", 1: "кликбейт"}
label2id = {"не кликбейт": 0, "кликбейт": 1}

In [27]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    "cointegrated/rubert-tiny2", num_labels=2, id2label=id2label, label2id=label2id
)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [28]:
split = tokenized_df.train_test_split(test_size = 0.2)
split

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2558
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 640
    })
})

In [29]:
from transformers import TrainingArguments, Trainer

In [30]:
training_args = TrainingArguments(
    output_dir="my_awesome_model",
    learning_rate=5e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=5,
    warmup_steps=100,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    remove_unused_columns=True,
    load_best_model_at_end=True,
    logging_steps=50
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=split['train'],
    eval_dataset=split['test'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss,Accuracy,Precision,F1
1,0.678500,0.454238,0.848437,0.818182,0.855869
2,0.276300,0.250856,0.900000,0.929766,0.896774
3,0.171300,0.241735,0.906250,0.936455,0.903226
4,0.116900,0.245945,0.917188,0.913580,0.917829
5,0.098500,0.253638,0.914062,0.920886,0.913658


TrainOutput(global_step=400, training_loss=0.26426276326179504, metrics={'train_runtime': 77.9527, 'train_samples_per_second': 164.074, 'train_steps_per_second': 5.131, 'total_flos': 3834904487976.0, 'train_loss': 0.26426276326179504, 'epoch': 5.0})

In [31]:
trainer.evaluate(split['test'])

{'eval_loss': 0.24173521995544434,
 'eval_accuracy': 0.90625,
 'eval_precision': 0.9364548494983278,
 'eval_f1': 0.9032258064516129,
 'eval_runtime': 0.4939,
 'eval_samples_per_second': 1295.88,
 'eval_steps_per_second': 40.496,
 'epoch': 5.0}

In [35]:
news_titles = pd.read_csv('C:/Users/slesa/Downloads/news_titles_test.csv', encoding='windows-1251', delimiter=';')
print(news_titles.head())

                                              titles  target
0  Фракция Порошенко запустила процедуру отставки...     NaN
1  Установленное при участии Трампа мирное соглаш...     NaN
2         Баффет написал последнее письмо акционерам     NaN
3  Нефтегазовые гиганты в кризисе. Почему бум ИИ ...     NaN
4  Клишас не исключил существование мошеннической...     NaN


In [36]:
test = []

for i in range(len(news_titles['titles'])):
  test.append(news_titles['titles'][i])

test

['Фракция Порошенко запустила процедуру отставки правительства Украины',
 'Установленное при участии Трампа мирное соглашение оказалось под угрозой',
 'Баффет написал последнее письмо акционерам',
 'Нефтегазовые гиганты в кризисе. Почему бум ИИ проходит мимо них',
 'Клишас не исключил существование мошеннической схемы с продажей жилья',
 'Саркози вышел на свободу. Первые кадры после освобождения',
 'Как запрет многократных шенгенских виз повлияет на поездки в ЕС',
 'Что может принести наибольшую доходность до конца года: топ-6 активов',
 'Сколько нужно зарабатывать для большой пенсии в России: расчеты',
 'Курс доллара ЦБ на 11 ноября снизился почти до ?81',
 'ЦБ назвал самый доходный актив в России с начала года',
 'Супругов из Майами признали самой долгоживущей парой в браке',
 'Два аэропорта ввели временные ограничения на полеты',
 '«Баланс» и «право на ошибку»: что известно о новой стратегии «Ростеха»',
 'Литва повторно попросит Белоруссию открыть коридор для застрявших фур',
 'В Бе

In [37]:
from transformers import pipeline

classifier = pipeline("text-classification", model=trainer.model.cpu(), tokenizer=tokenizer)
classifier(test)

Device set to use cpu


[{'label': 'не кликбейт', 'score': 0.9767279028892517},
 {'label': 'не кликбейт', 'score': 0.9288146495819092},
 {'label': 'не кликбейт', 'score': 0.93513023853302},
 {'label': 'не кликбейт', 'score': 0.9472692012786865},
 {'label': 'не кликбейт', 'score': 0.9586208462715149},
 {'label': 'не кликбейт', 'score': 0.6323631405830383},
 {'label': 'не кликбейт', 'score': 0.9199606776237488},
 {'label': 'кликбейт', 'score': 0.9336825013160706},
 {'label': 'кликбейт', 'score': 0.98479825258255},
 {'label': 'не кликбейт', 'score': 0.9400477409362793},
 {'label': 'не кликбейт', 'score': 0.7816800475120544},
 {'label': 'не кликбейт', 'score': 0.7360971570014954},
 {'label': 'не кликбейт', 'score': 0.9794700741767883},
 {'label': 'кликбейт', 'score': 0.9661126136779785},
 {'label': 'не кликбейт', 'score': 0.9467094540596008},
 {'label': 'не кликбейт', 'score': 0.9799922108650208},
 {'label': 'не кликбейт', 'score': 0.959179162979126},
 {'label': 'кликбейт', 'score': 0.5061095356941223},
 {'label'